<H2>Detecting nodes within a distance that may occur due to unjoined mesh</H2> 

In [48]:
import pandas as pd
import numpy as np
import itertools   
import time
import win32com.client as win32
lusas = win32.gencache.EnsureDispatch("Lusas.Modeller.19.0")
if not lusas.existsDatabase():
    Exception("A model must be open before running this code")


Define the distance below which we want to detect adjacent nodes

In [49]:
TOLERANCE = 0.05 # Distance between nodes which is suspicious, this should be smaller than the mesh size
APPROACH = 2     # Approach to be taken, 2 is reccommended
EXTENT_OPTION = 1 # 0=all nodes in database, 1=nodes in current selection (recommended)

# In general it is best to select lines to be checked since it is at lines that mesh cracks form, it is not worth checking all nodes within surfaces and volumes
try:
    lusas.setVisible(False) # Setting Lusas Modeller invisible speeds up calls to teh LPI because the application doesnt have to respond to user events at the same time.
    match EXTENT_OPTION:
        case 0: 
            nodes = lusas.database().getObjects("Nodes")
        case 1:
            nodes = lusas.selection().getObjects("Nodes")
finally:
    lusas.setVisible(True)

## Approach 1, using itertools to generate the necessary comparison combinations

Note, the limit of 100 nodes because this code is very slow for large number of nodes

In [50]:
if APPROACH == 1:
    if len(nodes) > 100:
        assert False, "This will take a very long time"

    start = time.time()
    no_comparisons = 0

    # Use itertools to generate the comparison of each node with each other node
    # No of comparisons will be n? where the ? indicates sumation of all integers fron n to 0
    for a, b in itertools.combinations(nodes, 2):
        no_comparisons+=1
        delta = np.sqrt( (a.getX() - b.getX())**2 + (a.getY() - b.getY())**2 + (a.getZ() - b.getZ())**2 )
        if delta < TOLERANCE:
            print(f"Nodes {a.getID()} and {b.getID()} are within {delta:.3f}")

    print(f"Comparison time = {time.time() - start:.2f} seconds for {no_comparisons} comparisons")


## Approach 2 - As 1 but reducing calls to the LPI

Same approach as above here but first we compile a list of node coordinates. 
This avoids many repeated calls to get node coordinates

In [51]:
if APPROACH == 2:
    start = time.time()
    no_comparisons = 0

    positions  = np.array([[n.getX(), n.getY(), n.getZ(), n.getID()] for n in nodes])
    for a, b in itertools.combinations(positions, 2):
        no_comparisons+=1
        delta = np.sqrt( (a[0] - b[0])**2 + (a[1] - b[1])**2 + (a[2] - b[2])**2 )
        if delta < TOLERANCE:
            print(f"Nodes {a[3]} and {b[3]} are within {delta:.3f}")

    print(f"Comparison time = {time.time() - start:.2f} seconds for {no_comparisons} comparisons")

Nodes 92721.0 and 123380.0 are within 0.001
Nodes 92722.0 and 123381.0 are within 0.002
Nodes 92723.0 and 123382.0 are within 0.004
Nodes 92724.0 and 123379.0 are within 0.005
Nodes 92725.0 and 128039.0 are within 0.004
Nodes 92726.0 and 128040.0 are within 0.003
Nodes 92727.0 and 128041.0 are within 0.003
Nodes 92728.0 and 128042.0 are within 0.002
Nodes 92729.0 and 128043.0 are within 0.001
Nodes 92820.0 and 128022.0 are within 0.010
Nodes 92821.0 and 128023.0 are within 0.019
Nodes 92822.0 and 128024.0 are within 0.029
Nodes 92823.0 and 128025.0 are within 0.038
Nodes 92824.0 and 128026.0 are within 0.048
Nodes 92834.0 and 127695.0 are within 0.029
Nodes 92848.0 and 127684.0 are within 0.025
Nodes 92857.0 and 127739.0 are within 0.029
Comparison time = 1.99 seconds for 17205 comparisons


As you can see repeated calls to the LPI functions can have a dramatic affect on performance and should be avoided where posible 

## Approach 3 using numpy arrays - more complicated

In [52]:
if APPROACH == 3:
    start = time.time()
    ids = np.array([n.getID() for n in nodes])
    positions  = np.array([[n.getX(), n.getY(), n.getZ()] for n in nodes])

    # Relative positions of each node 
    rel = positions[None, :, :] - positions[:, None, :]

    # distances of each node from the x,y,z relative offsets
    distances = np.sqrt(np.square(rel).sum(axis=2))


    for i in range(0, len(distances)):
        a = np.where(distances[i] < TOLERANCE)
        if len(a[0]) == 2 and a[0][0] != i:
            print(f"Nodes {ids[a[0][0]]} and {ids[a[0][1]]} are within {distances[i][a[0][0]]:.3f}")


    print(f"Comparison time = {time.time() - start:.2f} seconds using numpy arrays")